In [1]:
import pyodbc
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=192.168.0.242;'
                      'DATABASE=FundDB;'
                      'UID=worker;'
                      'PWD=worker;')
cursor = conn.cursor()

Filter out OEFs with geopraphical focus = Japan and add additional benchmark data from mstar&blg and filter out funds benchmarked with TOPIX

In [2]:
stock_name = 'Murata'
stock_isin = 'JP3914400001'
benchmark = 'TOPIX'

In [3]:
# additional_benchmark = pd.read_excel('data/Mstar_blg_unclassified_benchmark.xlsx', sheet_name=0, usecols="B:C")
# additional_benchmark = dict(zip(additional_benchmark['ISIN'], additional_benchmark['Primary Prospectus Benchmark']))

# additional_benchmark

In [4]:
# query_universe = "select lipper_id, e_name, launch_date, isin, domicile_id, price_ccy, fund_manager_benchmark from m_funds\
#                   where geographical_focus = 'Japan' and asset_universe = 'Mutual Funds' and asset_type = 'Equity'\
#                   and is_primary_fund = 1 and active = 1 and fund_of_funds = 0"

# df_universe = pd.read_sql(query_universe, conn)
# df_universe['lipper_id'] = df_universe['lipper_id'].apply(lambda x: int(x))

# for keys in additional_benchmark:
#     df_universe.at[df_universe[df_universe['isin']==keys].index, 'fund_manager_benchmark'] = additional_benchmark[keys]

# df_universe[df_universe['isin']=='LU1205057935']

In [7]:
df_universe = pd.read_excel('data/Japan_OEF_universe.xlsx')

df_Topix_funds = df_universe[df_universe['fund_manager_benchmark'].str.contains('Topix|TOPIX|TPX')].drop(['Unnamed: 0'], axis=1)  # Could be changed for different benchmark
TOPIX_funds_list = df_Topix_funds['lipper_id'].to_list()

df_Topix_funds

,lipper_id,e_name,launch_date,isin,domicile_id,price_ccy,fund_manager_benchmark
4,68373877,AMO Japan Stock Pick Concentrated Equity Ptf,2016-06-02,LU1375832596,LUX,JPY,Topix TR
10,40005319,Commonwealth Japan Fund,1989-07-10,US2030421067,USA,USD,Topix CR
18,40224738,BNY Mellon Japan Womenomics Fund;A,2018-09-13,US05587A4040,USA,USD,TOPIX TR JPY
21,60001423,Comgest Growth Japan JPY Acc,2000-05-19,IE0004767087,IRL,JPY,Topix NR
22,60001642,Nikko AM Shenton Japan SGD,1986-12-31,SG9999004279,SGP,SGD,Topix NR
...,...,...,...,...,...,...,...
703,62001911,AM-One MHAM Emerging Growth Stock Open,2000-02-25,JP90C00030Y3,JPN,JPY,Topix CR
714,62002905,AberdeenSTD Japan Small Capital Equity Fund,2000-01-27,JP90C00028X9,JPN,JPY,TPX
753,62903721,Daiwa iFreeNEXT Japanese Small Cap Equity Index,2018-10-19,JP90C000H0R7,JPN,JPY,Topix Small TR
778,60003657,LionGlobal Japan Growth SGD,1999-12-03,SG9999002331,SGP,SGD,Topix TR


Start combining 3yrs past holdings data and further filter out funds with frequent holding disclosures to backtest 

In [8]:
hldgs1_df = pd.read_csv('data/JP_MF_10yr/Mutual_funds_JP_20101031~20110331.csv')
hldgs2_df = pd.read_csv('data/JP_MF_10yr/Mutual_funds_JP_20110430~20110930.csv')
hldgs3_df = pd.read_csv('data/JP_MF_10yr/Mutual_funds_JP_20111031~20120331.csv')
hldgs4_df = pd.read_csv('data/JP_MF_10yr/Mutual_funds_JP_20120430~20120930.csv')
hldgs5_df = pd.read_csv('data/JP_MF_10yr/Mutual_funds_JP_20121031~20130331.csv')
hldgs6_df = pd.read_csv('data/JP_MF_10yr/Mutual_funds_JP_20130430~20130930.csv')

hldgs1_df = pd.read_csv('data/JP_MF_10yr/Mutual_funds_JP_20171031~20180228.csv')
hldgs2_df = pd.read_csv('data/JP_MF_10yr/Mutual_funds_JP_20180331~20180831.csv')
hldgs3_df = pd.read_csv('data/JP_MF_10yr/Mutual_funds_JP_20180930~20190228.csv')
hldgs4_df = pd.read_csv('data/JP_MF_10yr/Mutual_funds_JP_20190331~20190831.csv')
hldgs5_df = pd.read_csv('data/JP_MF_10yr/Mutual_funds_JP_20190930~20200229.csv')
hldgs6_df = pd.read_csv('data/JP_MF_10yr/Mutual_funds_JP_20200331~20200930.csv')

In [12]:
df_holdings = pd.concat([hldgs1_df, hldgs2_df, hldgs3_df, hldgs4_df, hldgs5_df, hldgs6_df], ignore_index=True)

df_holdings.head()

,LipperID,Date,Security,WeightCurrent,WeightChange,WeightPrevious,BookValueperShareMRQ,Currency,CUSIP,DividendPayoutRatio,...,PriceToCashFlow,PriceToDividend,PriceToEarnings,PriceToSales,RIC,SalesGrowth1Year,SalesGrowth3Year,SalesPerShareGrowth1Year,SalesPerShareGrowth3Year,SEDOL
0,40000027,2017/12/31,Aruhi Corp ORD,0.27,NaN,NaN,571.50,NaN,NaN,NaN,...,7.69,9999.00,14.47,2.21,7198.T,27.02,NaN,27.99,NaN,BF25643
1,40000027,2017/12/31,Astellas Pharma Inc ORD,0.76,-0.08,0.84,665.68,NaN,J03393105,NaN,...,11.92,41.06,16.10,2.29,4503.T,-4.45,4.79,-2.22,NaN,6985383
2,40000027,2017/12/31,Benefit One Inc ORD,1.12,-0.02,1.14,178.84,NaN,J0447X108,NaN,...,40.70,98.29,49.71,6.84,2412.T,13.15,13.14,13.92,NaN,B02JV67
3,40000027,2017/12/31,Calbee Inc ORD,1.18,-0.19,1.37,NaN,NaN,J05190103,NaN,...,18.94,87.26,27.76,1.98,2229.T,2.56,8.08,2.50,NaN,B3TBRZ8
4,40000027,2017/12/31,Central Japan Railway Co ORD,0.43,-0.02,0.45,14831.18,NaN,J05523105,NaN,...,6.49,144.14,9.96,2.32,9022.T,1.07,2.06,1.07,NaN,6183552


In [13]:
df_holdings_disclosure = df_holdings[['LipperID', 'Date']].drop_duplicates().set_index('LipperID')

diclose_freq = pd.Series(dict((fund, len(df_holdings_disclosure.loc[fund])) for fund in TOPIX_funds_list), name='# of months count')
min_date = pd.Series(dict((fund, df_holdings_disclosure.groupby('LipperID').get_group(fund).sort_values(by='Date').iloc[0][0]) for fund in TOPIX_funds_list), name='min_date')
max_date = pd.Series(dict((fund, df_holdings_disclosure.groupby('LipperID').get_group(fund).sort_values(by='Date').iloc[-1][0]) for fund in TOPIX_funds_list), name='max_date')

df_holdings_des = pd.concat([diclose_freq, min_date, max_date], axis=1)
df_holdings_des.head()

,# of months count,min_date,max_date
68373877,4,2018/12/31,2020/6/30
40005319,29,2017/10/31,2020/9/30
40224738,25,2018/10/31,2020/9/30
60001423,33,2017/10/31,2020/6/30
60001642,6,2017/12/31,2020/6/30


In [14]:
filt = (df_holdings_des['max_date'] >= '2020/8/31') & (df_holdings_des['# of months count'] >= 25)
df_funds_chosen = df_holdings_des[filt].sort_values('# of months count', ascending=False) # total 53 funds
# df_funds_chosen.to_excel(f'results/Chosen_{benchmark}_funds_DES.xlsx')

print(len(df_funds_chosen))
df_funds_chosen.head()

53


,# of months count,min_date,max_date
68131652,36,2017/10/31,2020/9/30
60011172,36,2017/10/31,2020/9/30
60033977,36,2017/10/31,2020/9/30
65146186,36,2017/10/31,2020/9/30
60052867,36,2017/10/31,2020/9/30


Get aggregated fund investment weight on stock chosen to backtest, count the different funds investing in the stock for each period

In [15]:
chosen_fund_list = list(df_funds_chosen.reset_index()['index'].unique())
filt2 = (df_holdings['ISIN'] == stock_isin) & (df_holdings['LipperID'].isin(chosen_fund_list)) 
df_stock_weightings = df_holdings[['LipperID', 'Date', 'Security', 'WeightCurrent', 'ISIN', 'MarketValueHeld']][filt2].drop_duplicates(subset=['LipperID','Date'])
df_stock_weightings['Date'] = pd.to_datetime(df_stock_weightings['Date'], format=r'%Y/%m/%d')

# Add Launch date of funds for bookkeeping and write to excel later
df_stock_weightings = df_stock_weightings.merge(df_Topix_funds[['lipper_id', 'launch_date']], how='left', left_on='LipperID', right_on='lipper_id').drop('lipper_id', axis=1)
df_stock_weightings.rename(columns={'launch_date':'Fund_LaunchDate'}, inplace=True)
df_stock_weightings

,LipperID,Date,Security,WeightCurrent,ISIN,MarketValueHeld,Fund_LaunchDate
0,60002658,2017-10-31,Murata Manufacturing Co Ltd ORD,2.04,JP3914400001,2.585859e+07,1996-04-30
1,60002658,2017-11-30,Murata Manufacturing Co Ltd ORD,1.28,JP3914400001,1.621515e+07,1996-04-30
2,60002658,2017-12-31,Murata Manufacturing Co Ltd ORD,0.87,JP3914400001,1.664921e+09,1996-04-30
3,60002859,2017-10-31,Murata Manufacturing Co Ltd ORD,3.21,JP3914400001,1.636959e+06,1985-07-31
4,60002859,2017-11-30,Murata Manufacturing Co Ltd ORD,2.72,JP3914400001,1.423168e+06,1985-07-31
...,...,...,...,...,...,...,...
698,68529751,2020-05-31,Murata Manufacturing Co Ltd ORD,1.16,JP3914400001,6.932856e+06,2019-04-10
699,68529751,2020-06-30,Murata Manufacturing Co Ltd ORD,1.21,JP3914400001,7.025387e+06,2019-04-10
700,68529751,2020-07-31,Murata Manufacturing Co Ltd ORD,1.31,JP3914400001,7.500901e+06,2019-04-10
701,68529751,2020-08-31,Murata Manufacturing Co Ltd ORD,0.96,JP3914400001,5.910275e+06,2019-04-10


In [16]:
date_list = sorted(df_stock_weightings['Date'].unique())
# print(date_list)

df_stock_weightings['FundAUM'] = df_stock_weightings['MarketValueHeld'] / (df_stock_weightings['WeightCurrent']/100)
date_grp = df_stock_weightings.groupby('Date')

fund_wgts = []
count = []
for date in date_list:
    grp = date_grp.get_group(date)
    cnt = len(grp)
    grp.drop(grp[grp['FundAUM']==np.inf].index, inplace=True)
    fund_wgt = sum(grp['MarketValueHeld']) / sum(grp['FundAUM']) * 100
    fund_wgts.append('{:3f}'.format(fund_wgt))
    count.append(cnt)

fund_wgts = pd.DataFrame({'FundPoolWeight':fund_wgts, 'Count': count}, index=date_list)
fund_wgts.head()

,FundPoolWeight,Count
2017-10-31,1.434064,14
2017-11-30,1.024715,12
2017-12-31,0.869676,14
2018-01-31,1.112208,10
2018-02-28,1.176770,11


In [17]:
df_benchmark = pd.read_csv('data/Benchmark/Nomura_NR_Topix_ETF_holdings_2017~2020.csv') # Change for different benchmark
df_benchmark = df_benchmark[df_benchmark['ISIN']==stock_isin][['LipperID', 'Date', 'Security', 'WeightCurrent', 'ISIN']]

## Add missing data if needed from mstar or blg
df_missing_benchmark = pd.read_excel('data/Benchmark/Nomura_NR_Topix_ETF_holdings_blg_20200331+20190430.xlsx')
# 'MURATA MANUFACTURING CO LTD'
# 'SONY CORP'
missing_data = pd.DataFrame([[62003319,'2019/4/30',stock_name,float('{:.3f}'.format(df_missing_benchmark[(df_missing_benchmark['Name']=='SONY CORP') & 
                                                                                   (df_missing_benchmark['Date']=='2019/4/30')]['% Wgt (P)'].values[0])), stock_isin]
                            ,[62003319,'2020/3/31',stock_name,float('{:.3f}'.format(df_missing_benchmark[(df_missing_benchmark['Name']=='SONY CORP') & 
                                                                                   (df_missing_benchmark['Date']=='2020/3/31')]['% Wgt (P)'].values[0])), stock_isin]]
                            ,columns=(['LipperID', 'Date', 'Security', 'WeightCurrent', 'ISIN']))

df_benchmark = df_benchmark.append(missing_data, ignore_index=True)
df_benchmark['Date'] = pd.to_datetime(df_benchmark['Date'], format=r'%Y/%m/%d')
df_benchmark = df_benchmark.sort_values('Date', ascending=True).set_index('Date')

df_benchmark.head()

,LipperID,Security,WeightCurrent,ISIN
Date,,,,
2017-10-31,62003319,Murata Manufacturing Co Ltd ORD,0.71,JP3914400001
2017-11-30,62003319,Murata Manufacturing Co Ltd ORD,0.63,JP3914400001
2017-12-31,62003319,Murata Manufacturing Co Ltd ORD,0.60,JP3914400001
2018-01-31,62003319,Murata Manufacturing Co Ltd ORD,0.61,JP3914400001
2018-02-28,62003319,Murata Manufacturing Co Ltd ORD,0.62,JP3914400001


In [14]:
df_backtest_final = pd.concat([df_benchmark, fund_wgts], axis=1)
df_benchmark['LipperID'] = df_benchmark['LipperID'].apply(lambda x: int(x))
df_backtest_final['Diff between fund weight and index weight on stock (%, RHS)'] = df_backtest_final['FundPoolWeight'].apply(lambda x: float(x)) \
                                                                        - df_backtest_final['WeightCurrent'].apply(lambda x: float(x))

df_backtest_final.rename(columns={'LipperID':'BenchmarkID', 'WeightCurrent':'Benchmark_StockWgt', 'Count':'FundCount', 'FundPoolWeight':'Avg fund invested weight in stock (%, RHS)'}, inplace=True)
df_backtest_final = df_backtest_final[['BenchmarkID', 'Security', 'ISIN', 'FundCount', 'Benchmark_StockWgt', 'Avg fund invested weight in stock (%, RHS)', 'Diff between fund weight and index weight on stock (%, RHS)']]

df_backtest_final.head()

,BenchmarkID,Security,ISIN,FundCount,Benchmark_StockWgt,"Avg fund invested weight in stock (%, RHS)","Diff between fund weight and index weight on stock (%, RHS)"
2017-10-31,62003319,Murata Manufacturing Co Ltd ORD,JP3914400001,14,0.71,1.434064,0.724064
2017-11-30,62003319,Murata Manufacturing Co Ltd ORD,JP3914400001,12,0.63,1.024715,0.394715
2017-12-31,62003319,Murata Manufacturing Co Ltd ORD,JP3914400001,14,0.60,0.869676,0.269676
2018-01-31,62003319,Murata Manufacturing Co Ltd ORD,JP3914400001,10,0.61,1.112208,0.502208
2018-02-28,62003319,Murata Manufacturing Co Ltd ORD,JP3914400001,11,0.62,1.176770,0.556770


In [18]:
# writer = pd.ExcelWriter(f'results/{stock_name}_backtest_results.xlsx', writer='xlsxwriter')

# df_backtest_final.to_excel(writer, sheet_name='final results')
# df_stock_weightings.to_excel(writer, sheet_name='selected funds raw data')

# writer.save()